<h1 align="center">Analítica de datos para la toma de decisiones empresariales</h1>
<h1 align="center">Ejemplo: Análisis de Serie de Tiempo</h1>
<h1 align="center">Centro de Educación Continua</h1>
<h1 align="center">EAFIT</h1>
<h1 align="center">2023</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

*** 
***Docente:*** Carlos Alberto Álvarez Henao, I.C. D.Sc.

***e-mail:*** calvar52@eafit.edu.co | carlosalvarezh@gmail.com

***Linkedin:*** https://www.linkedin.com/in/carlosalvarez5/

***github:*** https://github.com/carlosalvarezh/

***Herramienta:*** [Jupyter Notebook](http://jupyter.org/) | [GoogleColab](https://colab.research.google.com/)

***Kernel:*** Python 3.11
***

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/CFD_Applied/blob/master/figs/CC-BY.png?raw=true">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

## Introducción

En el mundo empresarial actual, la capacidad de tomar decisiones informadas y estratégicas es esencial para el éxito. Una valiosa herramienta para lograrlo es el análisis de datos de series temporales. En una amplia variedad de sectores, desde finanzas hasta marketing, las organizaciones recopilan información a lo largo del tiempo, como precios de acciones, consumo de energía y métricas de redes sociales. Este tipo de datos pueden revelar patrones, tendencias estacionales y pronósticos futuros que impulsan la generación de ganancias. Por ejemplo, al comprender cómo varía la demanda de productos a lo largo del año, las empresas pueden planificar estratégicamente promociones para optimizar las ventas. En este contexto, exploraremos un enfoque práctico utilizando el método ARIMA para analizar públicamente disponibles datos de pasajeros de aerolíneas en Python. A través de este ejemplo, aprenderemos a descomponer tendencias, identificar patrones y pronosticar eventos futuros, fortaleciendo así nuestras habilidades en la toma de decisiones basadas en datos.

## Carga de datos y análisis exploratorio

Empecemos el ejercicio con la importación de las bibliotecas a trabajar y la lectura de los datos

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
df = pd.read_csv("Data/AirPassengers.csv")

In [ ]:
df.head(10)

In [ ]:
df.tail()

Podemos observar que los datos contienen una columna etiquetada como "Month" que contiene fechas. En esa columna, las fechas están formateadas como año-mes. También vemos que los datos comienzan en enero de 1949 y van hasta diciembre de 1960.

La segunda columna está etiquetada como "#Passangers" y contiene el número de pasajeros para el año-mes.

Veamos cuántos elementos contiene el dataset y cuántos son no-nulos:

In [ ]:
df.info()

In [ ]:
null_data = df.isnull()
null_data

Si el dataset cuenta con valores faltantes y quisiéramos determinar en qué fila se encuentran, podemos ejecutar este código

In [ ]:
rows_with_missing_indices = df[df.isna().any(axis=1)].index
print(rows_with_missing_indices)

O si lo que necesitamos es saber la posición exacta del elemento nulo, podemos emplear este código:

In [ ]:
# Obtener la posición exacta de los valores nulos
rows, cols = np.where(df.isna())

# Crear una lista de tuplas con las posiciones
null_positions = list(zip(rows, cols))

print(null_positions)

En donde el primer elemento de la tupla corresponde a la fila y el segundo, a la columna.

Lo siguiente que desearemos hacer es convertir la columna `Month` en un objeto de fecha y hora. Esto permitirá extraer de manera programática valores temporales como el año o el mes para cada registro. Para lograrlo, utilizamos el método `to_datetime()`  de `Pandas`:

In [ ]:
df['Month'] = pd.to_datetime(df['Month'], format='%Y-%m')
df.head()

In [ ]:
df.info()

Este proceso automáticamente inserta el primer día de cada mes, lo cual es básicamente un valor ficticio dado que no tenemos datos diarios de pasajeros.

Observe también que el tipo de dato cambio de `object` a `datetime64[ns]`

Lo siguiente que podemos hacer es convertir la columna de meses en un índice. Esto nos permitirá trabajar de manera más sencilla con algunos de los paquetes que cubriremos más adelante:

In [ ]:
df.set_index('Month', inplace=True)
df

Grafiquemos los datos del DataFrame

In [ ]:
sns.lineplot(df)
plt.ylabel('Number of Passengers')

## Análisis de Estacionariedad

La estacionariedad es esencial en el análisis de series temporales, indicando que los cambios en los datos a lo largo del tiempo son constantes y no muestran tendencias ni patrones estacionales. Esto facilita la modelización y es un supuesto en muchos métodos de pronóstico. Utilizaremos la prueba Dickey Fuller para verificar la estacionariedad en nuestros datos, permitiéndonos aceptar o rechazar la hipótesis de no estacionariedad. Si rechazamos la hipótesis nula, confirmamos la existencia de estacionariedad, lo que es crucial para evaluar cómo los valores actuales se relacionan con los pasados en el conjunto de datos.

Calcularemos una media móvil de siete meses:

### Media Móvil

La [media móvil](https://en.wikipedia.org/wiki/Moving_average) se calcula como una técnica para suavizar las fluctuaciones en los datos y resaltar las tendencias subyacentes a lo largo del tiempo. Se utiliza en análisis de series temporales para reducir el ruido o variabilidad aleatoria en los datos, lo que facilita la identificación de patrones y tendencias más claras.

La idea detrás de la media móvil es tomar un conjunto de valores consecutivos de una serie temporal y calcular su promedio. A medida que avanzamos en el tiempo, se va ajustando la ventana de valores que se promedian, lo que suaviza las fluctuaciones a corto plazo y revela las tendencias a largo plazo.

La media móvil es especialmente útil para detectar patrones y tendencias en datos que pueden tener variaciones estacionales, cíclicas o aleatorias. Puede ser una herramienta efectiva para la exploración preliminar de datos y proporcionar una base para el análisis más profundo en el campo de la estadística y el análisis de series temporales.

Para el ejemplo, calculemos una media móvil de siete meses:

In [ ]:
rolling_mean = df.rolling(7).mean()
rolling_std = df.rolling(7).std()

A continuación, vamos a superponer nuestra serie temporal con la media móvil de 7 meses y la desviación estándar móvil de siete meses. En primer lugar, creemos una gráfica utilizando Matplotlib de nuestra serie temporal, junto con la media móvil a 7 meses y su desviación estándar:

In [ ]:
plt.plot(df, color="blue",label="Original Passenger Data")
plt.plot(rolling_mean, color="red", label="Rolling Mean Passenger Number")
plt.plot(rolling_std, color="black", label = "Rolling Standard Deviation in Passenger Number")

plt.title("Passenger Time Series, Rolling Mean, Standard Deviation")
plt.legend(loc="best")

### Pruebas estadísticas

A continuación, importemos la prueba aumentada [Dickey-Fuller](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test) del paquete statsmodels. La documentación de la prueba se puede encontrar [aquí](https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html). Luego, pasemos nuestro *DataFrame* al método `adfuller`. Aquí, especificamos el parámetro `autolag` como "`AIC`", lo que significa que se elige el rezago para minimizar el criterio de información. Finalmente, almacenemos nuestros resultados en un *DataFrame* y lo mostremos:

In [ ]:
adft = adfuller(df,autolag="AIC")

adftValues = [adft[0],adft[1],adft[2],adft[3], adft[4]['1%'], adft[4]['5%'], adft[4]['10%']]

metrics = ["Test Statistics","p-value","No. of lags used","Number of observations used",
           "critical value (1%)", "critical value (5%)", "critical value (10%)"]

output_df = pd.DataFrame({"Values": adftValues, "Metric": metrics})
print(output_df)

Podemos observar que nuestros datos no son estacionarios debido a que nuestro valor $p> 5\%$ y el estadístico de prueba es mayor al valor crítico. También podemos llegar a estas conclusiones al inspeccionar los datos, ya que podemos ver claramente una tendencia creciente en el número de pasajeros.

### Autocorrelación

Verificar la [autocorrelación](https://en.wikipedia.org/wiki/Autocorrelation) en los datos de series temporales en Python es otra parte importante del proceso analítico. Esto es una medida de cómo está correlacionada la serie temporal en un punto dado en el tiempo con los valores pasados, lo que tiene implicaciones significativas en diversas industrias. Por ejemplo, si nuestros datos de pasajeros tienen una fuerte autocorrelación, podemos asumir que un alto número de pasajeros hoy sugiere una alta probabilidad de que también sean altos mañana.

Pandas tiene un método de autocorrelación que podemos utilizar para calcular la autocorrelación en nuestros datos de pasajeros. Hagámoslo para un rezago de uno, tres, seis y nueve meses:

In [ ]:
autocorrelation_lag1 = df['#Passengers'].autocorr(lag=1)
print("One Month Lag: ", autocorrelation_lag1)

autocorrelation_lag3 = df['#Passengers'].autocorr(lag=3)
print("Three Month Lag: ", autocorrelation_lag3)

autocorrelation_lag6 = df['#Passengers'].autocorr(lag=6)
print("Six Month Lag: ", autocorrelation_lag6)

autocorrelation_lag9 = df['#Passengers'].autocorr(lag=9)
print("Nine Month Lag: ", autocorrelation_lag9)

Vemos que, incluso con un retraso de nueve meses, los datos presentan una autocorrelación alta. Esto es una evidencia adicional de las tendencias a corto y largo plazo en los datos.

### Descomposición

La [descomposición de tendencias](https://en.wikipedia.org/wiki/Decomposition_of_time_series) es otra manera útil de visualizar las tendencias en los datos de series temporales. Para continuar, importemos `seasonal_decompose` del paquete `statsmodels`. Pasemos nuestro *DataFrame* al método `season_decompose` y grafiquemos el resultado:

In [ ]:
decompose = seasonal_decompose(df['#Passengers'],model='additive', period=7)
decompose.plot()
plt.show()

En este gráfico, podemos ver claramente la tendencia creciente en el número de pasajeros y los patrones estacionales en el aumento y caída de los valores cada año.

### Pronóstico (Forecasting)

El [pronóstico de series temporales](https://en.wikipedia.org/wiki/Time_series) nos permite predecir valores futuros en una serie temporal dada la información actual y pasada. Aquí, utilizaremos el método [ARIMA](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average) para pronosticar el número de pasajeros, lo que nos permitirá pronosticar valores futuros en función de una combinación lineal de valores pasados. Utilizaremos el paquete `auto_arima`, lo que nos evitará el proceso de ajuste de hiperparámetros que consume mucho tiempo.

Primero, dividamos nuestros datos para entrenamiento y prueba, y visualicemos la división:

In [ ]:
df['Date'] = df.index
train = df[df['Date'] < pd.to_datetime("1960-08", format='%Y-%m')]
train['train'] = train['#Passengers']
del train['Date']
del train['#Passengers']
test = df[df['Date'] >= pd.to_datetime("1960-08", format='%Y-%m')]
del test['Date']
test['test'] = test['#Passengers']
del test['#Passengers']
plt.plot(train, color = "black")
plt.plot(test, color = "red")
plt.title("Train/Test split for Passenger Data")
plt.ylabel("Passenger Number")
plt.xlabel('Year-Month')
plt.grid(True)
sns.set()
plt.show()

La línea negra corresponde a nuestros datos de entrenamiento (`train`) y la línea roja corresponde a nuestros datos de prueba (`test`).

Importemos `auto_arima` del paquete `pdmarima`, entrenemos nuestro modelo y generemos predicciones.

<div class="alert alert alert-success">
$\color{red}{\textbf{Nota:}}$ Si es la primera vez que se va a ejecutar este paquete en su sistema (computador), debes instalarlo ejecutando la siguiente celda. Una vez instalado, no será necesario volverla a ejecutar, ni tampoco en ningún otro proyecto (notebook) que use este computador:
</div>


In [ ]:
pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima
model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train)
forecast = model.predict(n_periods=len(test))
forecast = pd.DataFrame(forecast,index = test.index,columns=['Prediction'])

Ahora, grafiquemos los resultados

In [ ]:
plt.plot(train, label='Train')
plt.plot(test, label='Test')
plt.plot(forecast, label='Prediction')
plt.title('#Passenger Prediction')
plt.xlabel('Date')
plt.ylabel('Actual #Passenger')
plt.legend(loc='upper left', fontsize=8)
plt.show()

Nuestras predicciones se muestran en verde y los valores reales se muestran en naranja.

Finalmente, calculemos el error cuadrático medio (RMSE):

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
rms = sqrt(mean_squared_error(test,forecast))
print("RMSE: ", rms)

Un valor de *RMSE* (*Root Mean Square Error*) de $64.4816^*$ indica la raíz del promedio de los errores al cuadrado entre los valores pronosticados y los valores reales en un conjunto de datos. Es una medida de la diferencia entre los valores pronosticados y los valores observados en una serie temporal. Cuanto menor sea el valor del RMSE, mejor será el ajuste del modelo a los datos observados.

En este contexto, un *RMSE* de $64.4816^*$ significa que, en promedio, los pronósticos del modelo difieren en alrededor de $64.4816^*$ unidades de la variable objetivo (en este caso, el número de pasajeros) de los valores reales en el conjunto de datos. Es importante interpretar este valor en relación con la escala de la variable y el contexto específico de la aplicación. Un *RMSE* más bajo indica que el modelo está haciendo predicciones más precisas y cercanas a los valores reales.

($^*$Este valor cambia cada vez que hacemos una nueva predicción)

### Análisis

El análisis de datos de series temporales desempeña un papel fundamental. Este análisis es una tarea que enfrentarán casi todos los profesionales en su carrera, y comprender las herramientas y métodos adecuados puede capacitar a los participantes para descubrir tendencias, anticipar eventos y, por ende, influir en la toma de decisiones estratégicas. A través de la comprensión de patrones estacionales mediante conceptos como estacionariedad, autocorrelación y descomposición de tendencias, se puede orientar la planificación de promociones a lo largo del año, mejorando así los resultados financieros de las empresas. Por último, la capacidad de prever eventos futuros mediante pronósticos de series temporales se convierte en una herramienta poderosa para guiar la toma de decisiones empresariales. Estos análisis se vuelven invaluables para cualquier profesional o equipo que busque aportar un valor significativo a sus organizaciones a través de la interpretación y aplicación de datos de series temporales